# Importing Packages

In [1]:
import numpy as np
import pandas as pd
import re
import requests
from collections import Counter
import bs4 as bs
import unicodedata
mdict = {"January":"1", "February":"2", "March":"3", "April":"4", "May":"5", "June":"6", "July":"7",\
         "August":"8", "September":"9", "October":"10", "November":"11", "December":"12"}
import us
STATES_DICT = us.states.mapping('name', 'abbr')

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)
def stripper(begins):
    begins = [strip_accents(boo) for boo in begins]
    begins = [re.sub(r"%C3%BA","u", u) for u in begins]
    begins = [re.sub(r"%C3%A9","e", e) for e in begins]
    begins = [re.sub(r"%C3%AD","i", i) for i in begins]
    begins = [re.sub(r"%C3%A1","a", a) for a in begins]
    begins = [re.sub("%27", "'", nra) for nra in begins]
    begins = [re.sub("%27", "'", nra) for nra in begins]
    begins = [re.sub(r"%22.*%22", "", nra) for nra in begins]
    begins = [re.sub(r" +", " ", nra) for nra in begins]
    #begins = [re.sub(" [A-Z]\. ", " ", nra) for nra in begins]
    return(begins)




# 116th Congress

In [11]:
source = requests.get("https://en.wikipedia.org/wiki/116th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link.get('href')) for link in soup.find_all('a')]
begins = stripper(begins)
ends = []
for string in begins:
    if re.match(r"/wiki", string):
        ends.append(re.sub(r"/wiki/","",string))
sens116 = ends[134:284]
reps116 = ends[294:1214]
####
####
####
sstate = []
schamber = []
sname = []
rstate = []
rchamber = []
rname = []
for sen in sens116:
    if re.match(r"List_of.*", sen):
        state = re.sub(r"^List_of_United_States_Senators_from_","",sen)
    else:
        name = re.sub(r"_"," ", sen)
        name = re.sub(r"\(.*\)","",name)
        sname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        sstate.append(state)
        schamber.append("Senate")
for dis in reps116:
    if re.match(r"List_of.*", dis):
        state = re.sub(r"^List_of_United_States_Representatives_from_","",dis)
    #elif re.match(r".*_congressional_district$", dis):
    #    house115.append(state)
    #    temp = re.sub(r"_congressional_district$","",dis)
    #    house115.append("DISTRICT: " + re.sub(r"^.*[^_]+_","", temp))
    elif not re.match(r".*_congressional_district$", dis):
        name = re.sub(r"_"," ",dis)
        name = re.sub(r"\(.*\)","",name)
        rname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        rstate.append(state)
        rchamber.append("House")
rstates116 = pd.concat([pd.Series(rname), pd.Series(rchamber), pd.Series(rstate)], axis = 1)
rstates116.columns = ["Name", "Chamber", "State"] 
sstates116 = pd.concat([pd.Series(sname), pd.Series(schamber), pd.Series(sstate)], axis = 1)
sstates116.columns = ["Name", "Chamber", "State"] 

In [12]:
sstates116.head()

,Name,Chamber,State
0,Doug Jones,Senate,Alabama
1,Richard Shelby,Senate,Alabama
2,Dan Sullivan,Senate,Alaska
3,Lisa Murkowski,Senate,Alaska
4,Kyrsten Sinema,Senate,Arizona


In [13]:
sstates116.tail()

,Name,Chamber,State
95,Shelley Moore Capito,Senate,West_Virginia
96,Tammy Baldwin,Senate,Wisconsin
97,Ron Johnson,Senate,Wisconsin
98,John Barrasso,Senate,Wyoming
99,Mike Enzi,Senate,Wyoming


In [14]:
rstates116.head()

,Name,Chamber,State
0,Bradley Byrne,House,Alabama
1,Martha Roby,House,Alabama
2,Mike Rogers,House,Alabama
3,Robert Aderholt,House,Alabama
4,Mo Brooks,House,Alabama


In [15]:
rstates116.tail()

,Name,Chamber,State
430,Jim Sensenbrenner,House,Wisconsin
431,Glenn Grothman,House,Wisconsin
432,Sean Duffy,House,Wisconsin
433,Mike Gallagher,House,Wisconsin
434,Liz Cheney,House,Wyoming


In [25]:
source = requests.get("https://en.wikipedia.org/wiki/116th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link) for link in soup.find_all('li')]
begins = stripper(begins)
#begins = [re.sub(r"\'","a",a) for a in begins]
sens116 = begins[217:317]
reps116 = begins[368:803]
syear = ["<BLANK>" for x in range(len(sens116))]
sday = ["<BLANK>" for x in range(len(sens116))]
smonth = ["<BLANK>" for x in range(len(sens116))]
sparty = ["<BLANK>" for x in range(len(sens116))]
sname = ["<BLANK>" for x in range(len(sens116))]
sstate = ["<BLANK>" for x in range(len(sens116))]
##########
ryear = ["<BLANK>" for x in range(len(reps116))]
rday = ["<BLANK>" for x in range(len(reps116))]
rmonth = ["<BLANK>" for x in range(len(reps116))]
rparty = ["<BLANK>" for x in range(len(reps116))]
rname = ["<BLANK>" for x in range(len(reps116))]
rstate = ["<BLANK>" for x in range(len(reps116))]
rdistrict = ["<BLANK>" for x in range(len(reps116))]
hmm = [re.sub(r"<[^>]+>","",sen) for sen in sens116]
for sen in range(len(hmm)):
    z = re.match("^3\..*",hmm[sen]) #class 3 begins this congress
    if z:
        syear[sen] = "2018" #election year
        smonth[sen] = "11" #election month
        sday[sen] = "6" #election day
        name = re.sub("^3\. ","",hmm[sen]) #Senate cycle according to wikipedia metadata
        sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(sname[sen],"",name)
        party = re.findall(r"\([^ ]+\)", party)[0]
        sparty[sen] = re.sub(r"[^A-Z]+","",party)
for sen in range(len(hmm)):
    if syear[sen] == "<BLANK>":
        if re.match("^[^0-9].*",hmm[sen]):
            step2 = re.findall(r"[A-Za-z]+ [0-9]+, \d\d\d\d",hmm[sen])[0]
            sday[sen] = re.sub(r",","",re.findall(r"[0-9]+,",step2)[0])
            syear[sen] = re.findall(r"\d\d\d\d", step2)[0]
            smonth[sen] = mdict[re.sub(r"[0-9, ]+","", step2)]
            name = re.sub(r"\[[^ ]+\]","",hmm[sen])
            sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
            party = re.sub(sname[sen],"",name)
            sparty[sen] = re.sub(r"[^A-Z]+","",party)
syear = [x for x in syear if x!= "<BLANK>"]
smonth = [x for x in smonth if x!= "<BLANK>"]
sday = [x for x in sday if x!= "<BLANK>"]
sname = [x for x in sname if x!= "<BLANK>"]
sparty = [x[0] for x in sparty if x!= "<BLANK>"]
schamber = ["Senate" for x in sparty if x!="<BLANK>"]
df = pd.DataFrame({'year': syear,
                       'month': smonth,
                       'day': sday})
sdate = pd.to_datetime(df)
senate116 = pd.concat([sdate, pd.Series(sparty), pd.Series(sname), pd.Series(schamber)], axis = 1)
senate116.columns = ["Date", "Party", "Name", "Chamber"]
#######
#######
#######
hmm = [re.sub(r"<[^>]+>","",rep) for rep in reps116]
hmm = [re.sub("[Aa]t\-large\.","0.",rep) for rep in hmm]
for rep in range(len(hmm)):
    z = re.match("^[0-9]+\..*",hmm[rep])
    if z:
        district = re.findall(r"^[0-9]+", hmm[rep])[0]
        rdistrict[rep] = district
        name = re.sub(rdistrict[rep] + ". ","", hmm[rep])
        rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(rname[rep] + " ","",name)
        party = re.findall(r"^\([^ ]+\)",party)[0]
        rparty[rep] = re.sub(r"[\(\)]","", party)
        ryear[rep] = "2018" #election year
        rmonth[rep] = "11" #election month
        rday[rep] = "6" #election day   
    else:
        if not re.search(".*Vacant.*", hmm[rep]):
            rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", hmm[rep])[0]
            rdistrict[rep] = district
            party = re.sub(rname[rep] + " ","", hmm[rep])
            party = re.findall(r"^\([^ ]+\)",party)[0]
            rparty[rep] = re.sub(r"[\(\)]","", party)
            date = re.sub("^.*from ","",hmm[rep])
            ryear[rep] = re.findall(r"\d\d\d\d",date)[0]
            date = re.sub(ryear[rep],"",date)
            date = re.sub(r",.*$","",date)
            rday[rep] = re.findall(r"[0-9]+",date)[0]
            date = re.sub(" " + rday[rep],"",date)
            rmonth[rep] = mdict[date]
ryear = [x for x in ryear if x!= "<BLANK>"]
rmonth = [x for x in rmonth if x!= "<BLANK>"]
rday = [x for x in rday if x!= "<BLANK>"]
rname = [x for x in rname if x!= "<BLANK>"]
rparty = [x[0] for x in rparty if x!= "<BLANK>"]
rchamber = ["House" for x in rparty if x!= "<BLANK>"]
rdistrict = [x for x in rdistrict if x!= "<BLANK>"]
df = pd.DataFrame({'year': ryear,
                       'month': rmonth,
                       'day': rday})
rdate = pd.to_datetime(df)
house116 = pd.concat([rdate, pd.Series(rparty), pd.Series(rname), pd.Series(rchamber), pd.Series(rdistrict)], axis = 1)
house116.columns = ["Date", "Party", "Name", "Chamber", "District"]  

In [26]:
senate116.head()

,Date,Party,Name,Chamber
0,2018-11-06,R,Richard Shelby,Senate
1,2018-11-06,R,Lisa Murkowski,Senate
2,2018-11-06,R,Jon Kyl,Senate
3,2018-11-06,R,John Boozman,Senate
4,2018-11-06,D,Kamala Harris,Senate


In [27]:
senate116.tail()

,Date,Party,Name,Chamber
29,2018-11-06,R,John Thune,Senate
30,2018-11-06,R,Mike Lee,Senate
31,2018-11-06,D,Patrick Leahy,Senate
32,2018-11-06,D,Patty Murray,Senate
33,2018-11-06,R,Ron Johnson,Senate


In [28]:
house116.head()

,Date,Party,Name,Chamber,District
0,2018-11-06,R,Bradley Byrne,House,1
1,2018-11-06,R,Martha Roby,House,2
2,2018-11-06,R,Mike Rogers,House,3
3,2018-11-06,R,Robert Aderholt,House,4
4,2018-11-06,R,Mo Brooks,House,5


In [29]:
house116.tail()

,Date,Party,Name,Chamber,District
430,2018-11-06,R,Jim Sensenbrenner,House,5
431,2018-11-06,R,Glenn Grothman,House,6
432,2018-11-06,R,Sean Duffy,House,7
433,2018-11-06,R,Mike Gallagher,House,8
434,2018-11-06,R,Liz Cheney,House,0


In [30]:
def hurry(x,y):
    return(pd.merge(x, y,  how='inner', left_on=['Name','Chamber'], right_on = ['Name','Chamber']))

In [31]:
senate_test = hurry(senate116,sstates116)
house_test = hurry(house116,rstates116)

In [32]:
#senate_test.to_csv("Senate_2018_test_set.csv")
#house_test.to_csv("House_2018_test_set.csv")